In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition\change format submission_Uncertainty challenge.ipynb
C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition\Kaggle M5 Accuracy .ipynb
C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition\M5 - Tuned Prediction.ipynb
C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition\M5 Forecasting-EDA, FE & Modelling.ipynb
C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition\M5-Competitors-Guide-Final-10-March-2020.doc
C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition\m5-first-public-notebook-under-0-50.ipynb
C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition\Naive Baseline - Quinn Wang Notebook - P1.ipynb
C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition\Others Naive Forecasts & Submission Score - QW - P2.ipynb
C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition\submission.csv
C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition\Top Down Distribution Method- QW -P3.ipynb
C:/Users/soreb_000/002 Kaggle/M5 Accuracy competition\Top Dow

	1. Calculate weight for the level 12 series
	2. Use the naive logic to make forecasts for each of the level 12 series
	3. Infer forecast, ground truth values, and weights for all the higher level series by aggregating
	4. Calculalte RMSSE for all series using the equation
	5. Multiply weight by respective RMSSE and add all these products

## 0. Import libraries and read in data

In [3]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tqdm import tqdm #tqdm is used to print the progress bar when wrapped around loops

In [4]:
df = pd.read_csv('../M5 Accuracy competition/submission.csv')

In [5]:
df.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,CA_1_FOODS_0.005_validation,2159.55790,1894.05440,1792.13370,1812.41100,2092.75830,2755.64940,2999.53640,2407.74460,2244.53440,...,2018.38000,2702.23900,3245.99050,2318.57890,2075.90450,1955.75010,1914.85890,2189.18300,2808.67360,3175.60550
1,CA_1_FOODS_1_0.005_validation,173.33507,161.68079,158.18413,175.89660,201.56816,237.31470,246.69475,178.03331,189.02340,...,214.15532,259.86804,232.60704,184.25563,178.40962,175.79298,199.19807,232.94340,305.94770,274.23172
2,CA_1_FOODS_2_0.005_validation,312.99520,273.68143,248.40652,246.91846,294.31094,406.16608,467.28770,399.91293,383.28442,...,277.87840,384.51242,488.84012,347.94640,290.62910,275.95163,253.55016,290.53064,394.38504,487.99768
3,CA_1_FOODS_3_0.005_validation,1491.05970,1270.82600,1197.87010,1209.70430,1395.79410,1978.10850,2105.43070,1642.74930,1484.85230,...,1375.08720,1906.95130,2339.21170,1525.84120,1374.90310,1314.11080,1277.57370,1487.74150,1993.48100,2302.94920
4,CA_1_HOBBIES_0.005_validation,320.91025,311.61930,296.88214,303.49963,352.43304,457.33130,459.61610,338.47820,345.28323,...,301.68475,416.30005,446.40982,314.62637,314.37213,299.76456,291.25160,335.89220,428.85004,458.36478


In [12]:
df_int=df.round().head()

In [11]:
df_int.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,CA_1_FOODS_0.005_validation,2160.0,1894.0,1792.0,1812.0,2093.0,2756.0,3000.0,2408.0,2245.0,...,2018.0,2702.0,3246.0,2319.0,2076.0,1956.0,1915.0,2189.0,2809.0,3176.0
1,CA_1_FOODS_1_0.005_validation,173.0,162.0,158.0,176.0,202.0,237.0,247.0,178.0,189.0,...,214.0,260.0,233.0,184.0,178.0,176.0,199.0,233.0,306.0,274.0
2,CA_1_FOODS_2_0.005_validation,313.0,274.0,248.0,247.0,294.0,406.0,467.0,400.0,383.0,...,278.0,385.0,489.0,348.0,291.0,276.0,254.0,291.0,394.0,488.0
3,CA_1_FOODS_3_0.005_validation,1491.0,1271.0,1198.0,1210.0,1396.0,1978.0,2105.0,1643.0,1485.0,...,1375.0,1907.0,2339.0,1526.0,1375.0,1314.0,1278.0,1488.0,1993.0,2303.0
4,CA_1_HOBBIES_0.005_validation,321.0,312.0,297.0,303.0,352.0,457.0,460.0,338.0,345.0,...,302.0,416.0,446.0,315.0,314.0,300.0,291.0,336.0,429.0,458.0


In [8]:
cal_df = pd.read_csv("../M5 Accuracy competition/m5-forecasting-accuracy/calendar.csv")

In [9]:
cal_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [10]:
cal_df["d"]=cal_df["d"].apply(lambda x: int(x.split("_")[1]))#split d_1 with separator "_" and take the second colomn[1] and convert to integer.
price_df["id"] = price_df["item_id"] + "_" + price_df["store_id"] + "_validation"#Create a ID in price_df that match with df id 

In [11]:
cal_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,5,NaN,NaN,NaN,NaN,1,0,1


## 1. Calculate weight for the level 12 series

In [12]:
for day in tqdm(range(1858, 1886)):#tqdm is used to print the progress bar when wrapped around loops
    wk_id = list(cal_df[cal_df["d"]==day]["wm_yr_wk"])[0] #For each day in this range, find the corresponding week id from cal_df 
    #Remember : tuto1: B= A[A<40] select element from A that are strict lower than 40.
    wk_price_df = price_df[price_df["wm_yr_wk"]==wk_id] #Use the week id to get the prices for that day from price_df
    df = df.merge(wk_price_df[["sell_price", "id"]], on=["id"], how='inner') #Merge the prices with df by the column ID, which is now on the same format in both DataFrames
    # select a.*,b.sell_price,b.id from df a inner join wk_price_df b on a.id=b.id
    df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)] # then create another column in df that stores the product prices and sell unit per day => and that will be the dollar sales by day
    df.drop(columns=["sell_price"], inplace=True)

100%|██████████████████████████████████████████| 28/28 [00:20<00:00,  1.37it/s]


In [13]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,unit_sales_1876,unit_sales_1877,unit_sales_1878,unit_sales_1879,unit_sales_1880,unit_sales_1881,unit_sales_1882,unit_sales_1883,unit_sales_1884,unit_sales_1885
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,24.78,8.26,24.78,8.26,16.52,16.52,0.00,8.26,8.26,8.26
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,3.97,3.97,3.97,3.97
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2.97,0.00,0.00,0.00,0.00,0.00,0.00,2.97,2.97,0.00
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,18.56,9.28,4.64,18.56,4.64,13.92,23.20,0.00,27.84,27.84
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8.64,5.76,5.76,5.76,8.64,2.88,0.00,0.00,0.00,0.00


In [14]:
df["dollar_sales"] = df[[c for c in df.columns if c.find("unit_sales")==0]].sum(axis=1) #total sales per line for column names that include the string "unit_sales"

In [22]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_1904,F_1905,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df.drop(columns=[c for c in df.columns if c.find("unit_sales")==0], inplace=True) #drop all columns that contain the string "unit_sales"

In [17]:
df["weight"] = df["dollar_sales"] / df["dollar_sales"].sum() #By default the axis is set to 0. here we have sum of rows.

In [18]:
df.drop(columns=["dollar_sales"], inplace=True)

## 2. Use the naive logic to make forecasts for each of the level 12 series
- All 0s
- Average through all history
- Same as previous 28 days
- Mean of previous 10, 20, 30, 40, 50, 60 days
- Average of same day for all previous weeks****

In [19]:
for d in tqdm(range(1886, 1914)):
    df["F_" + str(d)] = 0

100%|█████████████████████████████████████████| 28/28 [00:00<00:00, 147.27it/s]


## 3. Infer forecast, ground truth values, and weights for all the higher level series by aggregating

In [20]:
agg_df = pd.DataFrame(df[[c for c in df.columns if c.find("d_") == 0 or c.find("F_") == 0]].sum()).transpose()
agg_df["level"] = 1
agg_df["weight"] = 1/12
column_order = agg_df.columns

In [21]:
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913,level,weight
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,0,0,0,0,0,0,0,0,1,0.083333


In [23]:
level_groupings = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [24]:
df.groupby(by=level_groupings[11]).sum()

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
item_id         state_id                                                      
FOODS_1_001     CA          6    3    2    3    7    5    8    3    5     2   
                TX          0    1    2    2    0    4    0    0    4     3   
                WI          0    2    0    1    0    9    2    1    2     5   
FOODS_1_002     CA          3    3    4    4    3    3    0    2    1     1   
                TX          0    0    2    0    0    0    0    1    0     0   
...                       ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
HOUSEHOLD_2_515 TX          0    0    0    0    0    0    0    0    0     0   
                WI          0    0    0    0    0    0    0    0    0     0   
HOUSEHOLD_2_516 CA          0    2    0    1    1    1    0    2    1     2   
                TX          2    1    0    0    0    0    1    1    0     0   
                WI          0    1    0    2    0    1    1    1    0     0   

                          ...  F_1904  F_1905  F_1906  F_1907  F_1908  F_1909  \
item_id         state_id  ...                                                   
FOODS_1_001     CA        ...       0       0       0       0       0       0   
                TX        ...       0       0       0       0       0       0   
                WI        ...       0       0       0       0       0       0   
FOODS_1_002     CA        ...       0       0       0       0       0       0   
                TX        ...       0       0       0       0       0       0   
...                       ...     ...     ...     ...     ...     ...     ...   
HOUSEHOLD_2_515 TX        ...       0       0       0       0       0       0   
                WI        ...       0       0       0       0       0       0   
HOUSEHOLD_2_516 CA        ...       0       0       0       0       0       0   
                TX        ...       0       0       0       0       0       0   
                WI        ...       0       0       0       0       0       0   

                          F_1910  F_1911  F_1912  F_1913  
item_id         state_id                                  
FOODS_1_001     CA             0       0       0       0  
                TX             0       0       0       0  
                WI             0       0       0       0  
FOODS_1_002     CA             0       0       0       0  
                TX             0       0       0       0  
...                          ...     ...     ...     ...  
HOUSEHOLD_2_515 TX             0       0       0       0  
                WI             0       0       0       0  
HOUSEHOLD_2_516 CA             0       0       0       0  
                TX             0       0       0       0  
                WI             0       0       0       0  

[9147 rows x 1942 columns]

In [25]:
for level in tqdm(level_groupings):
    temp_df = df.groupby(by=level_groupings[level]).sum().reset_index(drop=True)
    temp_df["level"] = level
    temp_df["weight"] /= 12
    agg_df = agg_df.append(temp_df[column_order])

del temp_df

100%|██████████████████████████████████████████| 10/10 [00:17<00:00,  1.71s/it]


In [26]:
df["weight"] /= 12

In [27]:
print(df.shape[0], agg_df.shape[0], df.shape[0] + agg_df.shape[0])

30490 12350 42840


In [28]:
agg_df["weight"].sum() + df["weight"].sum()

0.9999999999999996

## 4. Calculalte RMSSE for all series using the equation

In [29]:
h = 28
n = 1885
def rmsse(ground_truth, forecast, train_series, axis=1): #ground truth = Y ; forecast = ^Y ; train_series = denominator
    # assuming input are numpy array or matrices
# The assert keyword is used when debugging code.
# The assert keyword lets you test if a condition in your code returns True, if not, the program will raise an AssertionError.
    assert axis == 0 or axis == 1
    assert type(ground_truth) == np.ndarray and type(forecast) == np.ndarray and type(train_series) == np.ndarray
    
    if axis == 1:
        # using axis == 1 we must guarantee these are matrices and not arrays
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis == 1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1]) ** 2).sum(axis=axis)
    return (1/h * numerator/denominator) ** 0.5

In [30]:
train_series_cols = [c for c in df.columns if c.find("d_") == 0][:-28] # until last 28 days
ground_truth_cols = [c for c in df.columns if c.find("d_") == 0][-28 # 28 days
forecast_cols = [c for c in df.columns if c.find("F_") == 0] # with the column name starting by F_

In [31]:
df["rmsse"] = rmsse(np.array(df[ground_truth_cols]), 
                   np.array(df[forecast_cols]), np.array(df[train_series_cols]))
agg_df["rmsse"] = rmsse(np.array(agg_df[ground_truth_cols]), 
                   np.array(agg_df[forecast_cols]), np.array(agg_df[train_series_cols]))

In [32]:
# for row_idx in range(len(df)):
#     row_df = pd.DataFrame(df.iloc[row_idx]).transpose()
#     train_series = np.array(row_df[train_series_cols].transpose()[row_idx])
#     ground_truth_series = np.array(row_df[ground_truth_cols].transpose()[row_idx])
#     forecast_series = np.array(row_df[forecast_cols].transpose()[row_idx])
#     print(rmsse(ground_truth_series, forecast_series, train_series, axis=0))

In [33]:
df["wrmsse"] = df["weight"] * df["rmsse"]
agg_df["wrmsse"] = agg_df["weight"] * agg_df["rmsse"]

In [34]:
df["wrmsse"].sum() + agg_df["wrmsse"].sum()

5.35979273095658